In [25]:
import numpy as np
from itertools import combinations
import json

In [26]:
np.random.seed(0)

In [27]:
with open('patterns.json', 'r') as f:
    patterns = json.load(f)
#patterns

In [28]:
N = len(patterns[0]) # Number of agents
s = np.sum([int(patterns[0][i][0]) for i in patterns[0]])  # Number of available spots
N,s

(7, np.int64(5))

In [29]:
neighbors = []
for pattern in patterns:
    neigh = np.array([[i for i in range(N)] for j in range(N)])
    neighbors.append(neigh)
#neighbors

In [30]:
def filter_pattern(pattern, f):
    f_pat = {}
    for i in pattern.keys():
        if f[int(i)]:
            f_pat[i] = pattern[i]
    return f_pat

def get_strategy(pattern, idx, known_idx, shuffle=False):
    #print(type(idx))
    my_neighb = known_idx[idx].tolist() 
    N = len(pattern)
    l = len(pattern[str(idx)])
    #print(my_neighb)

    for Nneigh in range(1,len(my_neighb)+1):
        neighb_lists = list(combinations(my_neighb, Nneigh))
        if shuffle:
            np.random.shuffle(neighb_lists)
        #print("neigh lists", neighb_lists)
        
        for n in neighb_lists:
            columns = sorted(n)
            #print(type(columns[0]))
            mask = [(i in columns) for i in range(N)]
            #print("mask", mask)
            f_pat = filter_pattern(pattern, mask) #Leave just the observed data columns
            #print("f_pat", f_pat) 
            
            #print(idx,pattern,f_pat)
            strat = {}
            in_freq = {}
            test = True
            for i in range(l):
                p = ''.join(f_pat[str(a)][i] for a in columns)
                #print(strat)
                if p not in strat:
                    strat[p] = pattern[str(idx)][(i+1)%l]
                    in_freq[p] = 1
                else:
                    if strat[p] != pattern[str(idx)][(i+1)%l]:
                        test = False
                        break
                    else:
                        in_freq[p] += 1
        
            if test: 
                same = True
                ini = strat[list(strat.keys())[0]]
                for p in strat:
                    same = same and (strat[p] == ini)
                if same: #If it takes always the same action...
                    return (), {'any': strat[list(strat.keys())[0]]}, {'any': 1}
                else:
                    return tuple(map(str,columns)), strat, in_freq
    return None  #If the output is None, then most likely there is no stable pattern

In [31]:
graphs = []
for n in range(len(patterns)):
    pat = patterns[n]
    neighbors_n = neighbors[n] 
    rel_data = {}
    for i in range(N): 
        rel_data[i] = {}
        rel_data[i]["pattern"] = pat[str(i)]
        strat = get_strategy(pat, i, neighbors_n, shuffle=True)
        #print(f"strat {i}", strat)
        rel_data[i]["neigh"] = strat[0]
        rel_data[i]["strat"] = strat[1]
        rel_data[i]["input freq"] = strat[2]
    graphs.append(rel_data)

In [32]:
with open(f'graph_data_N{N:d}s{s:d}.json', 'w') as json_file:
  json.dump(graphs, json_file)